<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/sir_demografico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp
import time

# Função que simula o modelo SIR demográfico discreto com N variável
def sir_disc(beta, gamma, nu, mu, alpha, N0, I0, R0, dt, T):
    passos = int(T / dt) + 1  # Número de passos de tempo, incluindo t=0
    S = np.zeros(passos)      # Vetor para suscetíveis
    I = np.zeros(passos)      # Vetor para infectados
    R = np.zeros(passos)      # Vetor para recuperados
    N = np.zeros(passos)      # Vetor para população total
    tempo = np.linspace(0, T, passos)  # Vetor de tempo

    # Condições iniciais
    S[0] = max(0, N0 - I0 - R0)  # Suscetíveis iniciais (garante S0 >= 0)
    I[0] = I0                    # Infectados iniciais
    R[0] = R0                    # Recuperados iniciais
    N[0] = S[0] + I[0] + R[0]    # População inicial (pode ser menor que N0 se I0 + R0 > N0)

    # Passo de tempo discreto
    for t in range(passos - 1):
        N[t] = S[t] + I[t] + R[t]  # Calcula população total no tempo t
        S[t+1] = S[t] + dt * (nu * N[t] - beta * S[t] * I[t] / N[t] - mu * S[t])  # Atualiza suscetíveis
        I[t+1] = I[t] + dt * (beta * S[t] * I[t] / N[t] - gamma * I[t] - (mu + alpha) * I[t])  # Atualiza infectados
        R[t+1] = R[t] + dt * (gamma * I[t] - mu * R[t])  # Atualiza recuperados
        N[t+1] = S[t+1] + I[t+1] + R[t+1]  # Atualiza população total

    return tempo, S, I, R, N  # Retorna tempo e todos os compartimentos, incluindo N

# Modelo SIR demográfico diferencial com N variável
def sir_diff(t, y, beta, gamma, nu, mu, alpha):
    S, I, R = y  # Desempacota variáveis
    N = S + I + R  # Calcula população total dinamicamente
    dS = nu * N - beta * S * I / N - mu * S  # Derivada de S (nascimentos - infecções - mortes)
    dI = beta * S * I / N - gamma * I - (mu + alpha) * I  # Derivada de I (infecções - recuperação - mortes)
    dR = gamma * I - mu * R  # Derivada de R (recuperação - mortes)
    return [dS, dI, dR]  # Retorna as derivadas

# Solucionador para equações diferenciais
def solve_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt, T):
    S0 = max(0, N0 - I0 - R0)  # Calcula S0 inicial (garante S0 >= 0)
    y0 = [S0, I0, R0]          # Condições iniciais
    intervalo = [0, T]         # Intervalo de tempo
    t_eval = np.arange(0, T + dt, dt)  # Pontos de tempo para avaliação
    t_eval = t_eval[t_eval <= T]       # Garante que não ultrapasse T

    sol = solve_ivp(sir_diff, intervalo, y0, args=(beta, gamma, nu, mu, alpha), t_eval=t_eval)
    N = sol.y[0] + sol.y[1] + sol.y[2]  # Calcula N(t) a partir das soluções
    return sol.t, sol.y[0], sol.y[1], sol.y[2], N  # Retorna tempo, S, I, R e N

# Widgets para parâmetros interativos
beta_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='beta')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='gamma')
nu_slider = widgets.FloatSlider(min=0.0015, max=0.1, step=0.001, value=0.01, description='nu')
mu_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='mu')
alpha_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='alpha')
N0_slider = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='N0')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=10, description='I0')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='R0')
T_slider = widgets.IntSlider(min=50, max=1000, step=50, value=365, description='T')
dt_disc_slider = widgets.FloatSlider(min=0.05, max=1.0, step=0.05, value=0.5, description='dt Disc')
dt_cont_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.02, value=0.1, description='dt Cont')

# Função de plotagem
def plot_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_disc, dt_cont, T):
    # Modelo discreto
    t_ini = time.process_time()  # Inicia medição de tempo
    t_disc, S_disc, I_disc, R_disc, N_disc = sir_disc(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_disc, T)
    cpu_time_disc = time.process_time() - t_ini  # Calcula tempo de CPU

    # Modelo contínuo
    t_ini_diff = time.process_time()  # Inicia medição de tempo
    t_diff, S_diff, I_diff, R_diff, N_diff = solve_sir(beta, gamma, nu, mu, alpha, N0, I0, R0, dt_cont, T)
    cpu_time_diff = time.process_time() - t_ini_diff  # Calcula tempo de CPU

    # Interpola solução contínua para pontos de tempo discretos
    I_diff_interp = np.interp(t_disc, t_diff, I_diff)
    diff_perc = (np.abs(I_disc - I_diff_interp) / N_disc) * 100  # Diferença percentual relativa a N(t)

    # Criação dos gráficos
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Gráfico de I(t) para ambos os modelos
    axs[0].plot(t_disc, I_disc, label='I (Discreto)', color='red', linestyle='dashed')
    axs[0].plot(t_diff, I_diff, label='I (Contínuo)', color='blue')
    axs[0].set_xlabel('Tempo (dias)')
    axs[0].set_ylabel('Indivíduos Infectados')
    axs[0].legend()
    axs[0].grid()

    # Gráfico da diferença percentual
    axs[1].plot(t_disc, diff_perc, label='Diferença (%)', color='purple')
    axs[1].set_xlabel('Tempo (dias)')
    axs[1].set_ylabel('ΔI/N(t) (%)')
    axs[1].grid()
    axs[1].legend()

    # Gráfico de N(t) para ambos os modelos
    axs[2].plot(t_disc, N_disc, label='N (Discreto)', color='red', linestyle='dashed')
    axs[2].plot(t_diff, N_diff, label='N (Contínuo)', color='blue')
    axs[2].set_xlabel('Tempo (dias)')
    axs[2].set_ylabel('População Total')
    axs[2].legend()
    axs[2].grid()

    plt.tight_layout()  # Ajusta o layout
    plt.show()

    # Exibe tempos de CPU
    print(f"Tempo de CPU (Discreto): {cpu_time_disc:.5f} s")
    print(f"Tempo de CPU (Contínuo): {cpu_time_diff:.5f} s")

# Interface interativa
ui = widgets.VBox([
    beta_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    N0_slider, I0_slider, R0_slider, T_slider, dt_disc_slider, dt_cont_slider
])
out = widgets.interactive_output(plot_sir, {
    'beta': beta_slider, 'gamma': gamma_slider, 'nu': nu_slider, 'mu': mu_slider, 'alpha': alpha_slider,
    'N0': N0_slider, 'I0': I0_slider, 'R0': R0_slider, 'T': T_slider,
    'dt_disc': dt_disc_slider, 'dt_cont': dt_cont_slider
})
display(ui, out)

Output()